# Slum Classification Using Satellite Images

In this notebook, we use [convolutional neural networks](http://cs231n.github.io/convolutional-networks/) to build an image classifier on satellite images of locations in India. The goal will be to classify images as slums vs non-slums. A good portion of this notebook is taken from [this tutorial](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)

Some terminology:
1. epoch: One forward pass and one backward pass of all the training examples
2. batch size:  the number of training examples in one forward/backward pass. The higher the batch size, the more memory space you'll need.

Images are supplied by [Paul Novosad](http://www.dartmouth.edu/~novosad/).

In [1]:
import glob

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.


## Data Preprocessing and Data Augmentation

In order to make the most of our few training examples, we will "augment" them via a number of random transformations, so that our model would never see twice the exact same picture. This helps prevent overfitting and helps the model generalize better.

In [2]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
)

Generate some pictures using this tool and save them to a temporary directory, so we can get a feel for what our augmentation strategy is doing.

In [3]:
img = load_img('../fake_data/train/india/2.png')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 900, 900)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 900, 900) -- (nb_sample, channel, height, width)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='../fake_data/preview', save_prefix='non_slum', save_format='png'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

# Building the Model

Let's start building the model! In our case we will use a relatively small convnet with a few layers and a few filters per layer, alongside data augmentation and dropout. Dropout also helps reduce overfitting, by preventing a layer from seeing twice the exact same pattern, thus acting in a way analoguous to data augmentation (you could say that both dropout and data augmentation tend to disrupt random correlations occuring in your data).

In [4]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 900, 900)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

On top of it we stick two fully-connected layers. We end the model with a single unit and a sigmoid activation, which is perfect for a binary classification. To go with it we will also use the binary_crossentropy loss to train our model.

In [5]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Let's prepare our data. We will use .flow_from_directory() to generate batches of image data (and their labels) directly from our pngs in their respective folders.

In [ ]:
batch_size = 160

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '../fake_data/train',  # this is the target directory
        target_size=(900, 900),  # all images will be resized to 900x900
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '../fake_data/validation',
        target_size=(900, 900),
        batch_size=batch_size,
        class_mode='binary')

Found 841 images belonging to 2 classes.
Found 250 images belonging to 2 classes.


We can now use these generators to train our model.

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=1000 // batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training

We get to about an 85% accuracy rate!

# Examining Error Rate

Let's look at prediction of slum pictures in validation set.

In [ ]:
def process_image(d):
    img = load_img(d)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)
    return x

directories = glob.glob("../data/validation/non_slum/*.png") + glob.glob("../data/validation/slum/*.png")
images = [process_image(d) for d in directories]

In [ ]:
[model.predict(i) for i in images]

Seems to always predict non-slum. Unfortunate..